In [4]:
import pandas as pd
import re
import tensorflow as tf

In [5]:
import wordpiece
import create_pretraining_data

In [7]:
a = pd.read_csv('../dataset/QnA_dataset.csv')

In [8]:
b = pd.read_csv('../dataset/paper.csv')

In [99]:
final = pd.read_csv('../dataset/finalDataset.csv')

In [100]:
final

,PAPER_FULL_TXT
0,목차먼지와 미세먼지미세먼지의 성분먼지와 미세먼지먼지란 대기 중에 떠다니거나 흩날려 ...
1,힘의 MKSA 단위. 질량 1 의 물체에 작용하여 1m/s2의 가속도( )를 발생시...
2,원래는 산소와 화합하는 반응 혹은 수소를 상실하는 반응을 산화라고 하였으나 현재는 ...
3,"지구상에 존재하는 수많은 쓰레기 중가장 고약하기로 소문난 #플라스틱오늘 리사이클맨,..."
4,"미세먼지 발생원은 자연적인 것과 인위적인 것으로 구분된다. 자연적 발생원은 흙먼지,..."
...,...
101197,지난 25일까지 애플워치 발화 발열 사고 15건 국내 14건 해외 1건 애플의 애플...
101198,오픈시그널 서울 인천 부산 조사 이전 전국 조사보다 가용성 상승 오픈시그널 홈페이지...
101199,불붙은 새 먹거리 전쟁 AI 혼합현실 기반 제조 금융 의료SK텔레콤 KT LG유플러...
101200,시대가 변하면 명당도 바뀐다. 게티이미지뱅크 제공강원도 삼척에는 조선 태조의 5대조...


In [101]:
final = final.dropna()
final = final.reset_index()
final = final.drop(['index'], axis='columns')

In [103]:
text = ''
for i in range(len(final)):
    #print(i)
    text += final.loc[i].values[0]

In [118]:
file = open('raw.txt', 'w')
file.write(text)
file.close()

In [6]:
file2 = open('my_vocab.txt', 'w')

In [7]:
!python wordpiece.py \
--corpus=raw.txt \
--iter=1000 \
--fname=my_vocab.txt

terminated vocabulary scanning
('##다', '##.')
('##으', '##로')
('##에', '##서')
('##o', '##n')
('##e', '##r')
('##t', '##i')
('##하', '##여')
('##하', '##였')
('##하', '##는')
('##e', '##n')
('##i', '##n')
('##e', '##s')
('연', '##구')
('##a', '##l')
('##q', '##u')
('##o', '##r')
('##하', '##고')
('##qu', '##o')
('##a', '##n')
('##하였', '##다.')
('##h', '##e')
('##0', '##0')
('있', '##다.')
('##ti', '##on')
('##e', '##d')
('##a', '##r')
('##s', '##quo')
('o', '##f')
('나', '##타')
('##i', '##g')
('대', '##한')
('##며', '##,')
('##l', '##e')
('##a', '##t')
('##적', '##으로')
('##n', '##d')
('사', '##용')
('t', '##he')
('##적', '##인')
('##i', '##c')
('있', '##는')
('##r', '##o')
('##r', '##e')
('##이', '##다.')
('##i', '##t')
('결', '##과')
('##0', '##1')
('경', '##우')
('##i', '##s')
('##a', '##s')
('a', '##nd')
('##in', '##g')
('분', '##석')
('##었', '##다.')
('##에서', '##는')
('것', '##으로')
('##s', '##p')
('##하', '##기')
('##en', '##t')
('##)', '##,')
('이', '##용')
('따', '##라')
('##e', '##l')
('##)', '##.')
('i', '##n')
('##a', '

In [9]:
file2.close()

In [8]:
f2 = open('my_vocab.txt', 'r')
kor_vocab = f2.read()
f2.close()

In [12]:
f1 = open('bert_vocab.txt', 'r')
bert_vocab = f1.read()
f1.close()

In [52]:
kor_vocab[31:35]

'##의\n'

In [93]:
final_vocab = bert_vocab + kor_vocab[31:]

In [94]:
vocab_list = final_vocab.split('\n')
len(vocab_list)

135548

In [95]:
# 중복 없애기
new_vocab = []
for x in vocab_list:
    if x not in new_vocab:
        new_vocab.append(x)
len(new_vocab)

131747

In [124]:
new_vocab.pop(new_vocab.index(''))

''

In [126]:
len(new_vocab)

131746

In [127]:
text = ''

for v in new_vocab:
    text +=  v+'\n'

In [129]:
print('기존 bert vocab.txt 크기:', len(bert_vocab))
print('새로 학습시킨 vocab.txt 크기:', len(kor_vocab))
print('중복 없애기 전 vocab.txt 크기:', len(final_vocab))
print('중복 없애기 후 vocab.txt 크기:', len(text))

기존 bert vocab.txt 크기: 764415
새로 학습시킨 vocab.txt 크기: 91953
중복 없애기 전 vocab.txt 크기: 856337
중복 없애기 후 vocab.txt 크기: 841571


In [128]:
f3 = open('final_vocab.txt', 'w')
f3.write(text)
f3.close()

In [8]:
!python create_pretraining_data.py \
--input_file=./training_data/Finalresult_30000.txt \
--vocab_file=final_vocab.txt \
--do_lower_case=False \
--max_seq_length=512 \
--output_file=./preprocessed_training_data/Finalresult_30000_tf.record



W1120 15:51:54.582713 47619347533312 module_wrapper.py:139] From create_pretraining_data.py:422: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1120 15:51:54.583001 47619347533312 module_wrapper.py:139] From create_pretraining_data.py:422: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1120 15:51:54.803255 47619347533312 module_wrapper.py:139] From create_pretraining_data.py:429: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W1120 15:51:54.807857 47619347533312 module_wrapper.py:139] From create_pretraining_data.py:431: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:*** Reading from input files ***
I1120 15:51:54.808015 47619347533312 create_pretraining_data.py:431] *** Reading from input files ***
INFO:tensorflow:  ./training_data/Finalresult_30000.txt
I1120 15:51:54.808149 47619347533312 create_p